1. Parsing the System of Equations :

In [29]:
import pathlib

# Function to parse the system of equations from a file
def parsing_system(path: pathlib.Path) -> tuple[list[list[float]], list[float]]:
    with path.open() as file:
        lines = file.readlines()
    matrixA = []
    vectorB = []
    variables = ['x', 'y', 'z']

    for line in lines:
        equation = line.split(' ')
        row = []
        for var in variables:
            for i in range(len(equation)):
                if equation[i].find(var) != -1:
                    coefficient = equation[i].replace(var, '')  # Extract the coefficient
                    if coefficient == '' and (equation[i-1] == '+' or i == 0):
                        row.append(1)
                    elif coefficient == '' and equation[i-1] == '-':
                        row.append(-1)
                    else:
                        row.append(int(coefficient))

        vectorB.append(int(equation[-1]))
        matrixA.append(row)

    return matrixA, vectorB

file_path = pathlib.Path("InputFile.txt")

A, B = parsing_system(file_path)

print(f"Matrix A: {A}")
print(f"Vector B: {B}")


Matrix A: [[2, 3, -1], [1, -1, 4], [3, 1, 2]]
Vector B: [5, 6, 7]


2. Matrix and Vector Operations:

a) Determinant: Write a function to compute the determinant of matrix
A. Recall one of the formulae for the determinant of a 3x3 matrix:

det(A) = a11(a22a33−a23a32)−a12(a21a33−a23a31)+a13(a21a32−a22a31)

In [30]:
def determinant(matrix: list[list[float]]) -> float:
    a = matrix[0][0] * (matrix[1][1] * matrix[2][2] - matrix[1][2] * matrix[2][1])
    b = matrix[0][1] * (matrix[1][0] * matrix[2][2] - matrix[1][2] * matrix[2][0])
    c = matrix[0][2] * (matrix[1][0] * matrix[2][1] - matrix[1][1] * matrix[2][0])
    return a - b + c

print(f"{determinant(A) = }")


determinant(A) = 14


b) Trace: Compute the sum of the elements along the main diagonal of
matrix A. For a matrix A, this is:

Trace(A) = a11 + a22 + a33

In [3]:
def trace(matrix: list[list[float]]) -> float:
    diagonal_sum = 0.0
    for i in range(len(matrix)):
        diagonal_sum += matrix[i][i]
    return diagonal_sum

print(f"{trace(A) = }")


trace(A) = 3.0


c) Vector norm: Compute the Euclidean norm of vector B, which is:

||B|| = √(b1^2 + b2^2 + b3^2)

In [5]:
def norm(vector: list[float]) -> float:
    return round(sum([v**2 for v in vector])**0.5, 2)

print(f"{norm(B) = }")


norm(B) = 10.49


d) Transpose of matrix: Write a function to compute the transpose of
matrix A. The transpose of a matrix A is obtained by swapping its rows
and columns.

In [24]:
def transpose_(matrix: list[list[float]]) -> list[list[float]]:
    result = []
    for col in range(len(matrix[0])): 
        new_row = []  
        for row in range(len(matrix)):  
            new_row.append(matrix[row][col])  
        result.append(new_row) 
    return result


def transpose(matrix: list[list[float]]) -> list[list[float]]:
    return [[matrix[row][col] for row in range(len(matrix))] for col in range(len(matrix[0]))]
#len(matrix[0]) is the number of columns
#len(matrix) is the number of rows

print(f"{transpose(A) = }")

transpose(A) = [[4, 3, 2], [7, 6, 5], [2, 1, 3]]


e) Matrix-vector multiplication: Write a function that multiplies matrix
A with vector B.

In [25]:
def multiply(matrix: list[list[float]], vector: list[float]) -> list[float]:
    return [sum(matrix[i][j] * vector[j] for j in range(len(vector))) for i in range(len(matrix))]

print(f"{multiply(A, B) = }")


multiply(A, B) =[76, 58, 61]


3. Solving using Cramer’s Rule

In [23]:
def solve_cramer(matrix: list[list[float]], vector: list[float]) -> list[float]:
    det_A = determinant(matrix)
    result = []
    for i in range(len(matrix)):
        new_matrix = [row.copy() for row in matrix] # Copy the matrix
        for j in range(len(matrix)):
            new_matrix[j][i] = vector[j]
        result.append(round(determinant(new_matrix) / det_A, 2))
    return result

print(f"{solve_cramer(A, B) = }")

solve_cramer(A, B)=[-4.0, 3.0, 0.0]


4. Solving using Inversion

In [26]:
def minor(matrix: list[list[float]], i: int, j: int) -> list[list[float]]:
    return [row[:j] + row[j+1:] for row in (matrix[:i] + matrix[i+1:])]

def cofactor(matrix: list[list[float]]) -> list[list[float]]:
    cofactors = []
    for i in range(len(matrix)):
        cofactor_row = []
        for j in range(len(matrix[i])):
            minor_matrix = minor(matrix, i, j)
            cofactor_value = ((-1) ** (i + j)) * determinant(minor_matrix) # (-1)^(i+j) * det(minor)
            cofactor_row.append(cofactor_value)
        cofactors.append(cofactor_row)
    return cofactors

def adjoint(matrix: list[list[float]]) -> list[list[float]]:
    return [list(row) for row in zip(*cofactor(matrix))]

def inverse(matrix: list[list[float]]) -> list[list[float]]:
    det = determinant(matrix)
    if det == 0:
        raise ValueError("The determinant of the matrix is 0, so the matrix is not invertible.")
    adj = adjoint(matrix)
    return [[adj[i][j] / det for j in range(len(matrix))] for i in range(len(matrix))]

def solve(matrix: list[list[float]], vector: list[float]) -> list[float]:
    inv_matrix = inverse(matrix)
    return [round(sum(inv_matrix[i][j] * vector[j] for j in range(len(vector))), 2) for i in range(len(inv_matrix))]

print(f"{solve(A, B) = }")

solve(A, B) = [-4.0, 3.0, -0.0]
